In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold, cross_val_score, train_test_split


import lightgbm as lgb
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool, cv

from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, f1_score

C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv("trainF.csv")
test = pd.read_csv("testF.csv")
test_id = pd.read_csv("test.csv")
test_id = test_id['id']
train.shape, test.shape

((78369, 66), (50226, 65))

In [3]:
target = train['redemption_status']
train.drop('redemption_status',axis = 1, inplace = True)

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train[train.columns]=sc.fit_transform(train)
test[test.columns]=sc.transform(test)

In [5]:
xgb_params = {
    'booster' : 'gbtree',
    'eta' : 0.1,
    'nthread' : 4,
    'silent' : True,
    'seed' : 1993,
    'scale_pos_weight': 106,
    'colsample_bytree': 0.547582701158404,
 'gamma': 7.99661203221007,
 'learning_rate': 0.036024836896883045,
 'max_delta_step': 9.545974437155495,
 'max_depth': 4,
 'min_child_weight': 3.9108711107457483,
 'n_estimators': 1015,
 'num_leaves': 100,
 'reg_alpha': 6.095828143522551,
 'reg_lambda': 2.9017833667247137,
 'subsample': 0.9458396927859323
}

In [64]:
err=[]
y_pred_tot1=[]
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
fold=StratifiedKFold(n_splits=10, shuffle=True, random_state=1993)
i=1

for train_index, test_index in fold.split(train,target):
    x_train, x_test = train[train_index], train[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    xgb = XGBClassifier(**xgb_params, random_state=1993)
    
    xgb.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric='auc', early_stopping_rounds=100, verbose=100)
    preds = xgb.predict_proba(x_test)[:,-1]
    
    print("ROC_AUC Score: ", roc_auc_score(y_test, preds))
    err.append(roc_auc_score(y_test,preds))
    p = xgb.predict_proba(test)[:,-1]
    print(f'--------------------Fold {i} completed !!!------------------')
    i=i+1
    y_pred_tot1.append(p)

[0]	validation_0-auc:0.809906
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.938055
[200]	validation_0-auc:0.942998
[300]	validation_0-auc:0.944431
[400]	validation_0-auc:0.944683
Stopping. Best iteration:
[345]	validation_0-auc:0.944996

ROC_AUC Score:  0.9449955184800942
--------------------Fold 1 completed !!!------------------
[0]	validation_0-auc:0.819718
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.925713
[200]	validation_0-auc:0.931412
[300]	validation_0-auc:0.932398
[400]	validation_0-auc:0.93355
[500]	validation_0-auc:0.933478
Stopping. Best iteration:
[461]	validation_0-auc:0.933829

ROC_AUC Score:  0.9338287706520434
--------------------Fold 2 completed !!!------------------
[0]	validation_0-auc:0.824563
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.939837
[200]	validation_0-auc:0.944916
[300]	validation_0-auc:0.945786
[400]	validation_0-auc:0.9

In [65]:
np.mean(err,0)

0.937864928489763

In [66]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = np.mean(y_pred_tot1,0)
submit.head()

,id,redemption_status
0,3,0.867842
1,4,0.031550
2,5,0.785191
3,8,0.001799
4,10,0.005568


In [67]:
submit.to_csv('xgboost_bayesianOptimization.csv',index=False)

In [68]:
xgb = XGBClassifier(**xgb_params, random_state=1993)
    
xgb.fit(train, target, verbose=100)
preds = xgb.predict_proba(test)[:,-1]

In [70]:
preds = xgb.predict_proba(test)[:,-1]
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = preds
submit.head()

,id,redemption_status
0,3,0.810154
1,4,0.002321
2,5,0.785862
3,8,0.000228
4,10,0.001312


In [71]:
submit.to_csv('xgboost_bayesianOptimization.csv',index=False)

In [6]:
lgb_params = {
     'metric': 'auc',
     'objective' : 'binary',
     'early_stopping_round':1000,
     'bagging_fraction': 0.4,
     'colsample_bytree': 0.4,
     'feature_fraction': 0.4,
     'lambda_l1': 0.0,
     'lambda_l2': 0.0,
     'learning_rate': 0.001,
     'max_depth': 15,
     'min_child_samples': 100,
     'min_child_weight': 50,
     'min_split_gain': 0.0010640577871197205,
     'n_estimators': 1520,
     'num_leaves': 196,
     'subsample': 0.4,
     'is_unbalance' :True
}


In [13]:
err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m=LGBMClassifier(**lgb_params, random_state=1993)
    m.fit(x_train,y_train,eval_set=[(x_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    i=i+1
    y_pred_tot.append(p)

C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.944333
[400]	valid_0's auc: 0.946806
[600]	valid_0's auc: 0.947563
[800]	valid_0's auc: 0.948256
[1000]	valid_0's auc: 0.948623
[1200]	valid_0's auc: 0.948882
[1400]	valid_0's auc: 0.949482
Did not meet early stopping. Best iteration is:
[1513]	valid_0's auc: 0.949685
err:  0.9496852349798508


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.930796
[400]	valid_0's auc: 0.932066
[600]	valid_0's auc: 0.933388
[800]	valid_0's auc: 0.934037
[1000]	valid_0's auc: 0.93436
[1200]	valid_0's auc: 0.934647
[1400]	valid_0's auc: 0.935113
Did not meet early stopping. Best iteration is:
[1518]	valid_0's auc: 0.935334
err:  0.9353337850140797


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.949893
[400]	valid_0's auc: 0.951668
[600]	valid_0's auc: 0.952577
[800]	valid_0's auc: 0.953126
[1000]	valid_0's auc: 0.95345
[1200]	valid_0's auc: 0.95375
[1400]	valid_0's auc: 0.953856
Did not meet early stopping. Best iteration is:
[1329]	valid_0's auc: 0.953948
err:  0.9539479720240238


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.931239
[400]	valid_0's auc: 0.930568
[600]	valid_0's auc: 0.930139
[800]	valid_0's auc: 0.92973
[1000]	valid_0's auc: 0.929083
[1200]	valid_0's auc: 0.929151
Early stopping, best iteration is:
[205]	valid_0's auc: 0.931336
err:  0.931335704657252


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.944272
[400]	valid_0's auc: 0.948715
[600]	valid_0's auc: 0.950573
[800]	valid_0's auc: 0.951403
[1000]	valid_0's auc: 0.9518
[1200]	valid_0's auc: 0.952026
[1400]	valid_0's auc: 0.952562
Did not meet early stopping. Best iteration is:
[1517]	valid_0's auc: 0.952779
err:  0.9527788511076765


In [14]:
m.best_iteration_

1517

In [15]:
m.best_score_

defaultdict(dict, {'valid_0': {'auc': 0.9527788511076765}})

In [16]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = np.mean(y_pred_tot,axis = 0)
submit.head()

,id,redemption_status
0,3,0.489110
1,4,0.055335
2,5,0.456539
3,8,0.010658
4,10,0.018004


In [17]:
submit.to_csv('lgbm_bo.csv',index=False)

In [18]:
lgb = LGBMClassifier(**lgb_params, random_state=1993)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [10]:
lgb = LGBMClassifier(**lgb_params, random_state=1993)
xgb = XGBClassifier(**xgb_params, random_state=1993)
gnb = GaussianNB()
cb = CatBoostClassifier(n_estimators=2500,random_state=1993,eval_metric='AUC',learning_rate=0.03)
lr = LogisticRegression(class_weight='balanced')

In [26]:
eclf = VotingClassifier(estimators=[('lr', lr), ('gnb', gnb), ('lgb', lgb),('xgb', xgb),('cb',cb)], voting='soft')

In [34]:
err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    eclf = VotingClassifier(estimators=[('lr', lr), ('gnb', gnb),('cb',cb), ('lgb', lgb),('xgb', xgb)], voting='soft')
    eclf.fit(x_train,y_train)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    i=i+1
    y_pred_tot.append(p)

0:	total: 93ms	remaining: 3m 52s
1:	total: 174ms	remaining: 3m 36s
2:	total: 258ms	remaining: 3m 34s
3:	total: 359ms	remaining: 3m 43s
4:	total: 448ms	remaining: 3m 43s
5:	total: 528ms	remaining: 3m 39s
6:	total: 626ms	remaining: 3m 43s
7:	total: 724ms	remaining: 3m 45s
8:	total: 803ms	remaining: 3m 42s
9:	total: 942ms	remaining: 3m 54s
10:	total: 1.07s	remaining: 4m 1s
11:	total: 1.16s	remaining: 3m 59s
12:	total: 1.24s	remaining: 3m 56s
13:	total: 1.35s	remaining: 3m 59s
14:	total: 1.46s	remaining: 4m 2s
15:	total: 1.57s	remaining: 4m 3s
16:	total: 1.68s	remaining: 4m 6s
17:	total: 1.77s	remaining: 4m 3s
18:	total: 1.88s	remaining: 4m 5s
19:	total: 1.96s	remaining: 4m 2s
20:	total: 2.09s	remaining: 4m 6s
21:	total: 2.18s	remaining: 4m 5s
22:	total: 2.28s	remaining: 4m 5s
23:	total: 2.37s	remaining: 4m 4s
24:	total: 2.48s	remaining: 4m 5s
25:	total: 2.57s	remaining: 4m 4s
26:	total: 2.66s	remaining: 4m 3s
27:	total: 2.74s	remaining: 4m 2s
28:	total: 2.84s	remaining: 4m 2s
29:	total: 2

239:	total: 22.9s	remaining: 3m 36s
240:	total: 23s	remaining: 3m 35s
241:	total: 23.1s	remaining: 3m 35s
242:	total: 23.2s	remaining: 3m 35s
243:	total: 23.3s	remaining: 3m 35s
244:	total: 23.3s	remaining: 3m 34s
245:	total: 23.4s	remaining: 3m 34s
246:	total: 23.5s	remaining: 3m 34s
247:	total: 23.6s	remaining: 3m 34s
248:	total: 23.7s	remaining: 3m 33s
249:	total: 23.7s	remaining: 3m 33s
250:	total: 23.8s	remaining: 3m 33s
251:	total: 23.9s	remaining: 3m 33s
252:	total: 24s	remaining: 3m 32s
253:	total: 24s	remaining: 3m 32s
254:	total: 24.1s	remaining: 3m 32s
255:	total: 24.2s	remaining: 3m 32s
256:	total: 24.3s	remaining: 3m 31s
257:	total: 24.4s	remaining: 3m 31s
258:	total: 24.4s	remaining: 3m 31s
259:	total: 24.5s	remaining: 3m 31s
260:	total: 24.6s	remaining: 3m 31s
261:	total: 24.8s	remaining: 3m 32s
262:	total: 25s	remaining: 3m 32s
263:	total: 25.1s	remaining: 3m 32s
264:	total: 25.2s	remaining: 3m 32s
265:	total: 25.3s	remaining: 3m 32s
266:	total: 25.4s	remaining: 3m 32s


470:	total: 43.6s	remaining: 3m 7s
471:	total: 43.7s	remaining: 3m 7s
472:	total: 43.8s	remaining: 3m 7s
473:	total: 43.9s	remaining: 3m 7s
474:	total: 43.9s	remaining: 3m 7s
475:	total: 44s	remaining: 3m 7s
476:	total: 44.1s	remaining: 3m 7s
477:	total: 44.2s	remaining: 3m 6s
478:	total: 44.3s	remaining: 3m 6s
479:	total: 44.3s	remaining: 3m 6s
480:	total: 44.4s	remaining: 3m 6s
481:	total: 44.5s	remaining: 3m 6s
482:	total: 44.6s	remaining: 3m 6s
483:	total: 44.7s	remaining: 3m 6s
484:	total: 44.7s	remaining: 3m 5s
485:	total: 44.8s	remaining: 3m 5s
486:	total: 44.9s	remaining: 3m 5s
487:	total: 45s	remaining: 3m 5s
488:	total: 45.1s	remaining: 3m 5s
489:	total: 45.1s	remaining: 3m 5s
490:	total: 45.2s	remaining: 3m 5s
491:	total: 45.3s	remaining: 3m 4s
492:	total: 45.4s	remaining: 3m 4s
493:	total: 45.5s	remaining: 3m 4s
494:	total: 45.5s	remaining: 3m 4s
495:	total: 45.6s	remaining: 3m 4s
496:	total: 45.7s	remaining: 3m 4s
497:	total: 45.8s	remaining: 3m 4s
498:	total: 45.9s	remain

703:	total: 1m 3s	remaining: 2m 40s
704:	total: 1m 3s	remaining: 2m 40s
705:	total: 1m 3s	remaining: 2m 40s
706:	total: 1m 3s	remaining: 2m 40s
707:	total: 1m 3s	remaining: 2m 40s
708:	total: 1m 3s	remaining: 2m 40s
709:	total: 1m 3s	remaining: 2m 40s
710:	total: 1m 3s	remaining: 2m 40s
711:	total: 1m 3s	remaining: 2m 39s
712:	total: 1m 3s	remaining: 2m 39s
713:	total: 1m 3s	remaining: 2m 39s
714:	total: 1m 3s	remaining: 2m 39s
715:	total: 1m 3s	remaining: 2m 39s
716:	total: 1m 4s	remaining: 2m 39s
717:	total: 1m 4s	remaining: 2m 39s
718:	total: 1m 4s	remaining: 2m 39s
719:	total: 1m 4s	remaining: 2m 38s
720:	total: 1m 4s	remaining: 2m 38s
721:	total: 1m 4s	remaining: 2m 38s
722:	total: 1m 4s	remaining: 2m 38s
723:	total: 1m 4s	remaining: 2m 38s
724:	total: 1m 4s	remaining: 2m 38s
725:	total: 1m 4s	remaining: 2m 38s
726:	total: 1m 4s	remaining: 2m 38s
727:	total: 1m 4s	remaining: 2m 38s
728:	total: 1m 5s	remaining: 2m 37s
729:	total: 1m 5s	remaining: 2m 37s
730:	total: 1m 5s	remaining:

927:	total: 1m 22s	remaining: 2m 19s
928:	total: 1m 22s	remaining: 2m 19s
929:	total: 1m 22s	remaining: 2m 19s
930:	total: 1m 22s	remaining: 2m 19s
931:	total: 1m 22s	remaining: 2m 18s
932:	total: 1m 22s	remaining: 2m 18s
933:	total: 1m 22s	remaining: 2m 18s
934:	total: 1m 22s	remaining: 2m 18s
935:	total: 1m 22s	remaining: 2m 18s
936:	total: 1m 22s	remaining: 2m 18s
937:	total: 1m 23s	remaining: 2m 18s
938:	total: 1m 23s	remaining: 2m 18s
939:	total: 1m 23s	remaining: 2m 18s
940:	total: 1m 23s	remaining: 2m 18s
941:	total: 1m 23s	remaining: 2m 17s
942:	total: 1m 23s	remaining: 2m 17s
943:	total: 1m 23s	remaining: 2m 17s
944:	total: 1m 23s	remaining: 2m 17s
945:	total: 1m 23s	remaining: 2m 17s
946:	total: 1m 23s	remaining: 2m 17s
947:	total: 1m 23s	remaining: 2m 17s
948:	total: 1m 23s	remaining: 2m 17s
949:	total: 1m 23s	remaining: 2m 17s
950:	total: 1m 24s	remaining: 2m 16s
951:	total: 1m 24s	remaining: 2m 16s
952:	total: 1m 24s	remaining: 2m 16s
953:	total: 1m 24s	remaining: 2m 16s
9

...

1382:	total: 1m 52s	remaining: 1m 31s
1383:	total: 1m 53s	remaining: 1m 31s
1384:	total: 1m 53s	remaining: 1m 31s
1385:	total: 1m 53s	remaining: 1m 30s
1386:	total: 1m 53s	remaining: 1m 30s
1387:	total: 1m 53s	remaining: 1m 30s
1388:	total: 1m 53s	remaining: 1m 30s
1389:	total: 1m 53s	remaining: 1m 30s
1390:	total: 1m 53s	remaining: 1m 30s
1391:	total: 1m 53s	remaining: 1m 30s
1392:	total: 1m 53s	remaining: 1m 30s
1393:	total: 1m 53s	remaining: 1m 30s
1394:	total: 1m 53s	remaining: 1m 30s
1395:	total: 1m 53s	remaining: 1m 30s
1396:	total: 1m 53s	remaining: 1m 29s
1397:	total: 1m 54s	remaining: 1m 29s
1398:	total: 1m 54s	remaining: 1m 29s
1399:	total: 1m 54s	remaining: 1m 29s
1400:	total: 1m 54s	remaining: 1m 29s
1401:	total: 1m 54s	remaining: 1m 29s
1402:	total: 1m 54s	remaining: 1m 29s
1403:	total: 1m 54s	remaining: 1m 29s
1404:	total: 1m 54s	remaining: 1m 29s
1405:	total: 1m 54s	remaining: 1m 29s
1406:	total: 1m 54s	remaining: 1m 29s
1407:	total: 1m 54s	remaining: 1m 29s
1408:	total:

1603:	total: 2m 10s	remaining: 1m 13s
1604:	total: 2m 11s	remaining: 1m 13s
1605:	total: 2m 11s	remaining: 1m 12s
1606:	total: 2m 11s	remaining: 1m 12s
1607:	total: 2m 11s	remaining: 1m 12s
1608:	total: 2m 11s	remaining: 1m 12s
1609:	total: 2m 11s	remaining: 1m 12s
1610:	total: 2m 11s	remaining: 1m 12s
1611:	total: 2m 11s	remaining: 1m 12s
1612:	total: 2m 11s	remaining: 1m 12s
1613:	total: 2m 11s	remaining: 1m 12s
1614:	total: 2m 11s	remaining: 1m 12s
1615:	total: 2m 11s	remaining: 1m 12s
1616:	total: 2m 11s	remaining: 1m 12s
1617:	total: 2m 12s	remaining: 1m 12s
1618:	total: 2m 12s	remaining: 1m 11s
1619:	total: 2m 12s	remaining: 1m 11s
1620:	total: 2m 12s	remaining: 1m 11s
1621:	total: 2m 12s	remaining: 1m 11s
1622:	total: 2m 12s	remaining: 1m 11s
1623:	total: 2m 12s	remaining: 1m 11s
1624:	total: 2m 12s	remaining: 1m 11s
1625:	total: 2m 12s	remaining: 1m 11s
1626:	total: 2m 12s	remaining: 1m 11s
1627:	total: 2m 12s	remaining: 1m 11s
1628:	total: 2m 12s	remaining: 1m 11s
1629:	total:

1825:	total: 2m 29s	remaining: 55.1s
1826:	total: 2m 29s	remaining: 55.1s
1827:	total: 2m 29s	remaining: 55s
1828:	total: 2m 29s	remaining: 54.9s
1829:	total: 2m 29s	remaining: 54.8s
1830:	total: 2m 29s	remaining: 54.7s
1831:	total: 2m 29s	remaining: 54.7s
1832:	total: 2m 29s	remaining: 54.6s
1833:	total: 2m 30s	remaining: 54.5s
1834:	total: 2m 30s	remaining: 54.4s
1835:	total: 2m 30s	remaining: 54.3s
1836:	total: 2m 30s	remaining: 54.2s
1837:	total: 2m 30s	remaining: 54.2s
1838:	total: 2m 30s	remaining: 54.1s
1839:	total: 2m 30s	remaining: 54s
1840:	total: 2m 30s	remaining: 53.9s
1841:	total: 2m 30s	remaining: 53.8s
1842:	total: 2m 30s	remaining: 53.7s
1843:	total: 2m 30s	remaining: 53.7s
1844:	total: 2m 30s	remaining: 53.6s
1845:	total: 2m 31s	remaining: 53.5s
1846:	total: 2m 31s	remaining: 53.4s
1847:	total: 2m 31s	remaining: 53.4s
1848:	total: 2m 31s	remaining: 53.3s
1849:	total: 2m 31s	remaining: 53.2s
1850:	total: 2m 31s	remaining: 53.1s
1851:	total: 2m 31s	remaining: 53.1s
1852:

2050:	total: 2m 47s	remaining: 36.7s
2051:	total: 2m 47s	remaining: 36.6s
2052:	total: 2m 47s	remaining: 36.5s
2053:	total: 2m 47s	remaining: 36.5s
2054:	total: 2m 47s	remaining: 36.4s
2055:	total: 2m 48s	remaining: 36.3s
2056:	total: 2m 48s	remaining: 36.2s
2057:	total: 2m 48s	remaining: 36.1s
2058:	total: 2m 48s	remaining: 36s
2059:	total: 2m 48s	remaining: 36s
2060:	total: 2m 48s	remaining: 35.9s
2061:	total: 2m 48s	remaining: 35.8s
2062:	total: 2m 48s	remaining: 35.7s
2063:	total: 2m 48s	remaining: 35.6s
2064:	total: 2m 48s	remaining: 35.5s
2065:	total: 2m 48s	remaining: 35.5s
2066:	total: 2m 48s	remaining: 35.4s
2067:	total: 2m 48s	remaining: 35.3s
2068:	total: 2m 49s	remaining: 35.2s
2069:	total: 2m 49s	remaining: 35.1s
2070:	total: 2m 49s	remaining: 35.1s
2071:	total: 2m 49s	remaining: 35s
2072:	total: 2m 49s	remaining: 34.9s
2073:	total: 2m 49s	remaining: 34.8s
2074:	total: 2m 49s	remaining: 34.7s
2075:	total: 2m 49s	remaining: 34.6s
2076:	total: 2m 49s	remaining: 34.6s
2077:	t

2277:	total: 3m 5s	remaining: 18.1s
2278:	total: 3m 6s	remaining: 18s
2279:	total: 3m 6s	remaining: 18s
2280:	total: 3m 6s	remaining: 17.9s
2281:	total: 3m 6s	remaining: 17.8s
2282:	total: 3m 6s	remaining: 17.7s
2283:	total: 3m 6s	remaining: 17.6s
2284:	total: 3m 6s	remaining: 17.6s
2285:	total: 3m 6s	remaining: 17.5s
2286:	total: 3m 6s	remaining: 17.4s
2287:	total: 3m 6s	remaining: 17.3s
2288:	total: 3m 6s	remaining: 17.2s
2289:	total: 3m 7s	remaining: 17.1s
2290:	total: 3m 7s	remaining: 17.1s
2291:	total: 3m 7s	remaining: 17s
2292:	total: 3m 7s	remaining: 16.9s
2293:	total: 3m 7s	remaining: 16.8s
2294:	total: 3m 7s	remaining: 16.7s
2295:	total: 3m 7s	remaining: 16.7s
2296:	total: 3m 7s	remaining: 16.6s
2297:	total: 3m 7s	remaining: 16.5s
2298:	total: 3m 7s	remaining: 16.4s
2299:	total: 3m 7s	remaining: 16.3s
2300:	total: 3m 7s	remaining: 16.2s
2301:	total: 3m 7s	remaining: 16.2s
2302:	total: 3m 8s	remaining: 16.1s
2303:	total: 3m 8s	remaining: 16s
2304:	total: 3m 8s	remaining: 15.9s


In [35]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = np.mean(y_pred_tot,axis = 0)
submit.head()

,id,redemption_status
0,3,0.559781
1,4,0.055534
2,5,0.522586
3,8,0.009507
4,10,0.018366


In [36]:
submit.to_csv('voting_classifier.csv',index=False)

In [11]:
err_lgb=[]
y_pred_test_lgb=[]
y_pred_trainCV_lgb = np.zeros(train.shape[0])
y_pred_train_lgb = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = LGBMClassifier(**lgb_params, random_state=1993)
    m.fit(x_train,y_train,eval_set=[(x_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_lgb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    y_pred_trainCV_lgb[test_index] = preds 
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_lgb.append(q)
    y_pred_test_lgb.append(p)

C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.944333
[400]	valid_0's auc: 0.946806
[600]	valid_0's auc: 0.947563
[800]	valid_0's auc: 0.948256
[1000]	valid_0's auc: 0.948623
[1200]	valid_0's auc: 0.948882
[1400]	valid_0's auc: 0.949482
Did not meet early stopping. Best iteration is:
[1513]	valid_0's auc: 0.949685
err:  0.9496852349798508


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.930796
[400]	valid_0's auc: 0.932066
[600]	valid_0's auc: 0.933388
[800]	valid_0's auc: 0.934037
[1000]	valid_0's auc: 0.93436
[1200]	valid_0's auc: 0.934647
[1400]	valid_0's auc: 0.935113
Did not meet early stopping. Best iteration is:
[1518]	valid_0's auc: 0.935334
err:  0.9353337850140797


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.949893
[400]	valid_0's auc: 0.951668
[600]	valid_0's auc: 0.952577
[800]	valid_0's auc: 0.953126
[1000]	valid_0's auc: 0.95345
[1200]	valid_0's auc: 0.95375
[1400]	valid_0's auc: 0.953856
Did not meet early stopping. Best iteration is:
[1329]	valid_0's auc: 0.953948
err:  0.9539479720240238


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.931239
[400]	valid_0's auc: 0.930568
[600]	valid_0's auc: 0.930139
[800]	valid_0's auc: 0.92973
[1000]	valid_0's auc: 0.929083
[1200]	valid_0's auc: 0.929151
Early stopping, best iteration is:
[205]	valid_0's auc: 0.931336
err:  0.931335704657252


C:\Users\kumarabh\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[200]	valid_0's auc: 0.944272
[400]	valid_0's auc: 0.948715
[600]	valid_0's auc: 0.950573
[800]	valid_0's auc: 0.951403
[1000]	valid_0's auc: 0.9518
[1200]	valid_0's auc: 0.952026
[1400]	valid_0's auc: 0.952562
Did not meet early stopping. Best iteration is:
[1517]	valid_0's auc: 0.952779
err:  0.9527788511076765


In [12]:
err_xgb=[]
y_pred_test_xgb=[]
y_pred_trainCV_xgb = np.zeros(train.shape[0])
y_pred_train_xgb = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = XGBClassifier(**xgb_params, random_state=1993)
    m.fit(x_train,y_train,eval_set=[(x_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_xgb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    y_pred_trainCV_xgb[test_index] = preds 
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_xgb.append(q)
    y_pred_test_xgb.append(p)

[0]	validation_0-auc:0.809906
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.942998
[400]	validation_0-auc:0.944683
[600]	validation_0-auc:0.94512
Stopping. Best iteration:
[585]	validation_0-auc:0.94561

err:  0.9456104041836928
[0]	validation_0-auc:0.819718
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.931412
[400]	validation_0-auc:0.93355
[600]	validation_0-auc:0.933105
Stopping. Best iteration:
[461]	validation_0-auc:0.933829

err:  0.9338287706520434
[0]	validation_0-auc:0.824563
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.944916
[400]	validation_0-auc:0.946217
[600]	validation_0-auc:0.9471
[800]	validation_0-auc:0.94817
[1000]	validation_0-auc:0.947707
[1014]	validation_0-auc:0.947538
err:  0.9485569153028024
[0]	validation_0-auc:0.811776
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.935877
[400]	validatio

In [13]:
err_cb=[]
y_pred_test_cb=[]
y_pred_trainCV_cb = np.zeros(train.shape[0])
y_pred_train_cb = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = CatBoostClassifier(n_estimators=2500,random_state=1993,eval_metric='AUC',learning_rate=0.05, 
                           bagging_temperature = 0.27, colsample_bylevel = 0.34, max_depth = 12, reg_lambda = 0.105)
    m.fit(x_train,y_train,eval_set=[(x_test, y_test)], early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_cb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    y_pred_trainCV_cb[test_index] = preds 
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_cb.append(q)
    y_pred_test_cb.append(p)

0:	test: 0.6892295	best: 0.6892295 (0)	total: 263ms	remaining: 10m 58s
200:	test: 0.9261237	best: 0.9301183 (83)	total: 1m 11s	remaining: 13m 35s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9301182839
bestIteration = 83

Shrink model to first 84 iterations.
err:  0.9301182838954642
0:	test: 0.7262903	best: 0.7262903 (0)	total: 84.5ms	remaining: 3m 31s
200:	test: 0.9238883	best: 0.9272176 (71)	total: 1m 19s	remaining: 15m 13s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9272176466
bestIteration = 71

Shrink model to first 72 iterations.
err:  0.927217646602161
0:	test: 0.7525619	best: 0.7525619 (0)	total: 101ms	remaining: 4m 13s
200:	test: 0.9377669	best: 0.9453952 (40)	total: 1m 25s	remaining: 16m 13s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9453951501
bestIteration = 40

Shrink model to first 41 iterations.
err:  0.9453951500779855
0:	test: 0.7572534	best: 0.7572534 (0)	total: 82ms	remaining: 3m 24s
200:	test

In [16]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = 0.4*np.mean(y_pred_test_lgb,axis = 0)+ 0.3*np.mean(y_pred_test_cb,axis = 0) + 0.3*np.mean(y_pred_test_xgb,axis = 0)
submit.head()

,id,redemption_status
0,3,0.464528
1,4,0.030495
2,5,0.433947
3,8,0.004749
4,10,0.008714


In [46]:
submit.to_csv('ensemble_weight.csv',index=False)

In [17]:
err=[]
y_pred_test_lr=[]
y_pred_trainCV_lr = np.zeros(train.shape[0])
y_pred_train_lr = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = LogisticRegression(class_weight='balanced')
    m.fit(x_train,y_train)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    y_pred_trainCV_lr[test_index] = preds 
    p = m.predict_proba(test)[:,-1]
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_lr.append(q)
    y_pred_test_lr.append(p)

err:  0.8655032358690973
err:  0.8809662439217181
err:  0.8807192310135292
err:  0.8894026169253244
err:  0.8574400287946475
err:  0.8612749041942792
err:  0.8589591581800089
err:  0.8685115002152541
err:  0.859760185753707
err:  0.8618552865075276


In [48]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = np.mean(y_pred_test_lr,axis = 0)
submit.head()

,id,redemption_status
0,3,0.916735
1,4,0.267561
2,5,0.718915
3,8,0.162817
4,10,0.097169


In [49]:
submit.to_csv('logistic.csv',index=False)

In [18]:
err=[]
y_pred_test_nb=[]
y_pred_trainCV_nb = np.zeros(train.shape[0])
y_pred_train_nb = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = GaussianNB()
    m.fit(x_train,y_train)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    y_pred_trainCV_nb[test_index] = preds 
    p = m.predict_proba(test)[:,-1]
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_nb.append(q)
    y_pred_test_nb.append(p)

err:  0.7579291143528615
err:  0.7737291185873685
err:  0.7431886190566929
err:  0.7481491675664993
err:  0.6952345916876628
err:  0.706249603014969
err:  0.7919022111184039
err:  0.7695607757616819
err:  0.7396722138708334
err:  0.7311818435514339


In [7]:
# from sklearn.neighbors import KNeighborsClassifier
# err=[]
# y_pred_test_knn=[]
# y_pred_trainCV_knn = np.zeros(train.shape[0])
# y_pred_train_knn = []
# from sklearn.model_selection import KFold,StratifiedKFold
# fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
# i=1
# for train_index, test_index in fold.split(train,target):
#     x_train, x_test = train.iloc[train_index], train.iloc[test_index]
#     y_train, y_test = target[train_index], target[test_index]
#     m = KNeighborsClassifier(n_neighbors = 5)
#     m.fit(x_train,y_train)
#     preds=m.predict_proba(x_test)[:,-1]
#     print("err: ",roc_auc_score(y_test,preds))
#     err.append(roc_auc_score(y_test,preds))
#     y_pred_trainCV_knn[test_index] = preds 
#     p = m.predict_proba(test)[:,-1]
#     q = m.predict_proba(train)[:,-1]
#     i=i+1
#     y_pred_train_knn.append(q)
#     y_pred_test_knn.append(p)

err:  0.6875666052663152
err:  0.7070347511874264
err:  0.6719103978319324
err:  0.6584852462718696
err:  0.721083610340666
err:  0.6736721291806935
err:  0.7200064576231712
err:  0.7394746035442823
err:  0.7066483524239023
err:  0.7681634252103726


In [19]:
y_pred_trainCV_cb.shape, y_pred_trainCV_knn.shape, y_pred_trainCV_lgb.shape, y_pred_trainCV_lr.shape, y_pred_trainCV_nb.shape, y_pred_trainCV_xgb.shape

((78369,), (78369,), (78369,), (78369,), (78369,), (78369,))

In [53]:
trainCV_rslt = pd.DataFrame({'y_pred_trainCV_cb': y_pred_trainCV_cb, 'y_pred_trainCV_knn': y_pred_trainCV_knn,
                            'y_pred_trainCV_lgb': y_pred_trainCV_lgb, 'y_pred_trainCV_lr': y_pred_trainCV_lr,
                            'y_pred_trainCV_nb': y_pred_trainCV_nb, 'y_pred_trainCV_xgb': y_pred_trainCV_xgb,
                            'y_pred_trainCV_rf': y_pred_trainCV_rf,'y_pred_trainCV_ada': y_pred_trainCV_ada})
trainCV_rslt.to_csv('trainCV_rslt.csv', index = False)

In [54]:
train_rslt = pd.DataFrame({'y_pred_train_cb': np.mean(y_pred_train_cb,0), 'y_pred_train_knn': np.mean(y_pred_train_knn,0),
                            'y_pred_train_lgb': np.mean(y_pred_train_lgb,0), 'y_pred_train_lr': np.mean(y_pred_train_lr,0),
                            'y_pred_train_nb': np.mean(y_pred_train_nb,0), 'y_pred_train_xgb': np.mean(y_pred_train_xgb,0),
                          'y_pred_train_rf': np.mean(y_pred_train_rf,0),'y_pred_train_ada': np.mean(y_pred_train_ada,0)})
train_rslt.to_csv('train_rslt.csv', index = False)

In [55]:
test_rslt = pd.DataFrame({'y_pred_test_cb': np.mean(y_pred_test_cb,0), 'y_pred_test_knn': np.mean(y_pred_test_knn,0),
                            'y_pred_test_lgb': np.mean(y_pred_test_lgb,0), 'y_pred_test_lr': np.mean(y_pred_test_lr,0),
                            'y_pred_test_nb': np.mean(y_pred_test_nb,0), 'y_pred_test_xgb': np.mean(y_pred_test_xgb,0),
                         'y_pred_test_rf': np.mean(y_pred_test_rf,0),'y_pred_test_ada': np.mean(y_pred_test_ada,0)})
test_rslt.to_csv('test_rslt.csv', index = False)

In [33]:
test_rslt.head()

,y_pred_test_cb,y_pred_test_knn,y_pred_test_lgb,y_pred_test_lr,y_pred_test_nb,y_pred_test_xgb
0,0.036757,0.00,0.489110,0.916735,6.868271e-03,0.859524
1,0.000893,0.00,0.055335,0.267561,6.339423e-04,0.026978
2,0.058017,0.18,0.456539,0.718915,3.298306e-03,0.779755
3,0.000157,0.00,0.010658,0.162817,6.032675e-08,0.001460
4,0.000455,0.00,0.018004,0.097169,1.870962e-05,0.004585


In [34]:
rf_params = {'max_depth': 5,
             'min_samples_leaf': 0.1,
             'min_samples_split': 0.1,
             'n_estimators': 100}

In [35]:
err_rf=[]
y_pred_test_rf=[]
y_pred_trainCV_rf = np.zeros(train.shape[0])
y_pred_train_rf = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = RandomForestClassifier(**rf_params, random_state=1993)
    m.fit(x_train,y_train)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_rf.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    y_pred_trainCV_rf[test_index] = preds 
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_rf.append(q)
    y_pred_test_rf.append(p)

err:  0.8870868709110542
err:  0.8711924724580606
err:  0.8889138842426938
err:  0.8952647625500201
err:  0.8607649989766608
err:  0.8498038011757815
err:  0.848671952742902
err:  0.8744830372707191
err:  0.8909508585462937
err:  0.8904613171904516


In [39]:
test_rslt.head()

,y_pred_test_cb,y_pred_test_knn,y_pred_test_lgb,y_pred_test_lr,y_pred_test_nb,y_pred_test_xgb,y_pred_test_rf
0,0.036757,0.00,0.489110,0.916735,6.868271e-03,0.859524,0.020914
1,0.000893,0.00,0.055335,0.267561,6.339423e-04,0.026978,0.017772
2,0.058017,0.18,0.456539,0.718915,3.298306e-03,0.779755,0.019927
3,0.000157,0.00,0.010658,0.162817,6.032675e-08,0.001460,0.010166
4,0.000455,0.00,0.018004,0.097169,1.870962e-05,0.004585,0.004862


In [45]:
test_rslt['y_pred_test_nb'][0]

0.006868270772956253

In [47]:
mega_y_pred = np.mean(test_rslt,1)
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = mega_y_pred
submit.head()

,id,redemption_status
0,3,0.332844
1,4,0.052739
2,5,0.316636
3,8,0.026466
4,10,0.017871


In [48]:
submit.to_csv('model_blend.csv',index=False)

In [51]:
ada_params = {'learning_rate': 1.01329,
              'n_estimators': 407}

In [52]:
err_ada=[]
y_pred_test_ada=[]
y_pred_trainCV_ada = np.zeros(train.shape[0])
y_pred_train_ada = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(train,target):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = AdaBoostClassifier(**ada_params, random_state=1993)
    m.fit(x_train,y_train)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_rf.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:,-1]
    y_pred_trainCV_ada[test_index] = preds 
    q = m.predict_proba(train)[:,-1]
    i=i+1
    y_pred_train_ada.append(q)
    y_pred_test_ada.append(p)

err:  0.925856429040249
err:  0.9235856746628274
err:  0.9307905118813209
err:  0.9280954246151891
err:  0.8907391331964176
err:  0.9183093025061224
err:  0.9092439287755922
err:  0.9237550549427281
err:  0.9054496340680203
err:  0.9423335623103785


In [56]:
trainCV_rslt.head()

,y_pred_trainCV_cb,y_pred_trainCV_knn,y_pred_trainCV_lgb,y_pred_trainCV_lr,y_pred_trainCV_nb,y_pred_trainCV_xgb,y_pred_trainCV_rf,y_pred_trainCV_ada
0,0.001742,0.0,0.069349,0.811695,0.139930,0.005052,0.013292,0.492974
1,0.001060,0.0,0.024957,0.392150,0.194216,0.001185,0.016781,0.492095
2,0.009751,0.0,0.253442,0.528410,0.000005,0.654660,0.016656,0.496893
3,0.000446,0.0,0.030161,0.329618,0.006794,0.006727,0.005751,0.490906
4,0.005055,0.0,0.258462,0.321966,0.001905,0.427252,0.011148,0.497068


In [58]:
trainX = trainCV_rslt[['y_pred_trainCV_cb','y_pred_trainCV_lgb','y_pred_trainCV_lr','y_pred_trainCV_rf','y_pred_trainCV_ada']]

In [61]:
testX = test_rslt[['y_pred_test_cb','y_pred_test_lgb','y_pred_test_lr','y_pred_test_rf','y_pred_test_ada']]

In [68]:
trainX.head()

,a,b,c,d,e
0,0.001742,0.069349,0.811695,0.013292,0.492974
1,0.001060,0.024957,0.392150,0.016781,0.492095
2,0.009751,0.253442,0.528410,0.016656,0.496893
3,0.000446,0.030161,0.329618,0.005751,0.490906
4,0.005055,0.258462,0.321966,0.011148,0.497068


In [69]:
testX.head()

,a,b,c,d,e
0,0.036757,0.489110,0.916735,0.020914,0.499193
1,0.000893,0.055335,0.267561,0.017772,0.494984
2,0.058017,0.456539,0.718915,0.019927,0.498596
3,0.000157,0.010658,0.162817,0.010166,0.470640
4,0.000455,0.018004,0.097169,0.004862,0.489250


In [74]:
testX.to_csv('testX.csv', index = False)
trainX.to_csv('trainX.csv', index = False)

In [67]:
trainX.columns = ['a','b','c','d','e']
testX.columns = ['a','b','c','d','e']

In [75]:
xgb_params = {
    'booster' : 'gbtree',
    'eta' : 0.1,
    'nthread' : 4,
    'silent' : True,
    'seed' : 1993,
    'scale_pos_weight': 106,
    'colsample_bytree': 0.5246884658540595,
 'gamma': 0.9601535611794482,
 'learning_rate': 0.039066082460214306,
 'max_delta_step': 0.6124301463312987,
 'max_depth': 1,
 'min_child_weight': 19.922110062205082,
 'n_estimators': 2324,
 'num_leaves': 37,
 'reg_alpha': 0.48975141190259586,
 'reg_lambda': 1.5283443574008326,
 'subsample': 0.41995227297967597}

In [76]:
err_xgbX=[]
y_pred_test_xgbX=[]
y_pred_trainCV_xgbX = np.zeros(train.shape[0])
y_pred_train_xgbX = []
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
i=1
for train_index, test_index in fold.split(trainX,target):
    x_train, x_test = trainX.iloc[train_index], trainX.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    m = XGBClassifier(**xgb_params, random_state=1993)
    m.fit(x_train,y_train,eval_set=[(x_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(x_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err_xgbX.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(testX)[:,-1]
    y_pred_trainCV_xgbX[test_index] = preds 
    q = m.predict_proba(trainX)[:,-1]
    i=i+1
    y_pred_train_xgbX.append(q)
    y_pred_test_xgbX.append(p)

[0]	validation_0-auc:0.869927
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.942501
[400]	validation_0-auc:0.943593
[600]	validation_0-auc:0.945018
[800]	validation_0-auc:0.946246
[1000]	validation_0-auc:0.946486
[1200]	validation_0-auc:0.946979
[1400]	validation_0-auc:0.947474
[1600]	validation_0-auc:0.9477
[1800]	validation_0-auc:0.947651
Stopping. Best iteration:
[1742]	validation_0-auc:0.947991

err:  0.9479914321808416
[0]	validation_0-auc:0.862005
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.931268
Stopping. Best iteration:
[138]	validation_0-auc:0.935176

err:  0.9351758731906303
[0]	validation_0-auc:0.835923
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.946533
Stopping. Best iteration:
[172]	validation_0-auc:0.94677

err:  0.9467704826632226
[0]	validation_0-auc:0.863082
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validat

In [77]:
submit = pd.DataFrame(columns = ['id','redemption_status'])
submit['id'] = test_id
submit['redemption_status'] = np.mean(y_pred_test_xgbX,0)
submit.head()

,id,redemption_status
0,3,0.923045
1,4,0.159802
2,5,0.925389
3,8,0.014304
4,10,0.024674


In [78]:
submit.to_csv('model_stacking.csv',index=False)